In [4]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import date, datetime
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle

In [5]:
# Read the csv files into a pandas DataFrame
train = pd.read_csv('master_train.csv')
test = pd.read_csv('master_test.csv')

In [6]:
test.head()

,Unnamed: 0,id,belongs_to_collection_True,budget,language_enc,popularity,status_Released,year,month,Action,...,Western,count_genre,count_country,count_company,count_splang,count_cast,count_crew,count_staff,count_keyword,castWt
0,0,3001,1,0,0,3.851534,1,2007.0,7.0,0,...,0,4,2,0,2,7,2,9,2,120
1,1,3002,0,88000,1,3.559789,1,2058.0,5.0,0,...,0,2,1,1,1,10,8,18,8,185
2,2,3003,0,0,1,8.085194,1,1997.0,5.0,0,...,0,2,1,3,1,9,10,19,4,128
3,3,3004,0,6800000,2,8.596012,1,2010.0,9.0,0,...,0,3,2,3,3,23,69,92,15,611
4,4,3005,0,2000000,1,3.217680,1,2005.0,2.0,0,...,0,2,1,0,1,4,14,18,4,84


In [7]:
train.head()

,Unnamed: 0,id,belongs_to_collection_True,budget,language_enc,popularity,status_Released,year,month,Action,...,count_genre,count_country,count_company,count_splang,count_cast,count_crew,count_staff,count_keyword,castWt,revenue
0,0,1,1,14000000,0,6.575393,1,2015,2,0,...,1,1,3,1,24,72,96,4,597,12314651
1,1,2,1,40000000,0,8.248895,1,2004,8,0,...,4,1,1,1,20,9,29,4,385,95149435
2,2,3,0,3300000,0,64.299990,1,2014,10,0,...,1,1,3,1,51,64,115,12,1270,13092000
3,3,4,0,1200000,1,3.174936,1,2012,3,0,...,2,1,0,2,7,3,10,7,151,16000000
4,4,5,0,0,2,1.148070,1,2009,2,1,...,2,1,0,1,4,2,6,0,80,3923970


In [8]:
# Assign the data to X and y
Xid = train[['id']]
X_train = train[['belongs_to_collection_True', 'budget', 'language_enc', 'popularity', 'status_Released','year', 'month', 'count_genre',
       'count_country', 'count_company', 'count_splang', 'count_cast',
       'count_crew', 'count_staff', 'count_keyword', "castWt"]]
y_train = train['revenue'].values.reshape(-1,1)
print(X_train.shape, y_train.shape)

(3000, 16) (3000, 1)


In [9]:
testID = test['id']
testID.head()
test = test[['belongs_to_collection_True', 'budget', 'language_enc', 'popularity', 'status_Released','year', 'month', 'count_genre',
       'count_country', 'count_company', 'count_splang', 'count_cast',
       'count_crew', 'count_staff', 'count_keyword', "castWt"]]

## GradientBoostingRegressor - No Scaling

class sklearn.ensemble.GradientBoostingRegressor(loss=’ls’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)[source]¶

In [10]:
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'} 
gbr = ensemble.GradientBoostingRegressor(**params)
predictions2 = gbr.fit(X_train,y_train)
training_score = gbr.score(X_train, y_train)
print(f"Training Score: {training_score}")

Training Score: 0.8625831275807061


In [11]:
feature_names = X_train.columns

In [12]:
sorted(zip(gbr.feature_importances_, feature_names), reverse=True)

[(0.36211973245914114, 'budget'),
 (0.19192643862529712, 'popularity'),
 (0.057123830969215826, 'count_staff'),
 (0.04967055295277423, 'belongs_to_collection_True'),
 (0.049551280394369304, 'month'),
 (0.04870616874755888, 'count_keyword'),
 (0.045963313897982506, 'year'),
 (0.04508924706126956, 'count_cast'),
 (0.041834852297561245, 'count_crew'),
 (0.03623533326745089, 'castWt'),
 (0.029165287432868486, 'count_company'),
 (0.015883463749701565, 'count_country'),
 (0.013953229428854332, 'count_splang'),
 (0.0116553272503614, 'count_genre'),
 (0.0011219414655930678, 'language_enc'),
 (0.0, 'status_Released')]

In [13]:
#Make New Predictions
gbr_predictions = pd.DataFrame(gbr.predict(test), columns = ['revenue'])
output = pd.concat([testID, gbr_predictions], axis=1)
output.head()
#rf_predictions = pd.DataFrame(rf.predict(test), columns = ['revenue'])
#output = pd.concat([testID['id'], rf_predictions], axis=1)

,id,revenue
0,3001,1.781335e+07
1,3002,3.872070e+06
2,3003,1.445405e+07
3,3004,3.068542e+07
4,3005,4.646821e+06


In [14]:
output.to_csv("outputs/output_" + str(date.today()) + "_6" + ".csv", index=False)